# Advent of Code 2015 - Day 5 - Naughty and Nice Strings

https://adventofcode.com/2015/day/5

We have to test strings for various patterns.  Here we assume that as will read each string character by character, using a number of separate string tests, and if any of the tests returns a 'Naughty' result, then the string is naughty and not nice.

In [1]:
type StringCategorisation = | Naughty | Nice

let defaultStringCategorisation = Nice

In [2]:
let stringInputFilePath = @"sample-input.txt"

stringInputFilePath

sample-input.txt

In [3]:
let identity x = x

// This is the signture we require for string-checking functions
type StringChecker = string -> Map<char,int> -> StringCategorisation

// Combine multiple StringCheckers into a single checking function
let createNaughtyOrNice (checkers: StringChecker list) =
    let naughtyOrNice (s: string): StringCategorisation =
        // charMap is a map of how many times each character occurs in 's'
        let charMap = s |> Seq.groupBy identity |> Seq.map (fun (s,sseq) -> (s, sseq |> Seq.toList |> List.length)) |> Map.ofSeq
        let results = checkers |> List.map (fun checker -> checker s charMap)
        if (results |> List.contains Naughty) then Naughty else Nice
    naughtyOrNice

let vowels = "aeiou" // can assume that all strings are lower-case only

// String checker #1 - check for at least three vowels
let vowelChecker: StringChecker = fun (s: string) (charMap: Map<char,int>) ->
    let vowelCount =
        vowels |> Seq.map (fun vowel ->
            if (charMap |> Map.containsKey vowel)
            then charMap[vowel]
            else 0
        ) |> Seq.sum
    if (vowelCount >= 3)
    then Nice
    else Naughty

// String checker #2 - check for at least one double
let doubleChecker: StringChecker = fun (s: string) (charMap: Map<char,int>) ->
    let isNice = // for any letter that occurs at least twice, search for a double
        charMap |> Map.keys |> Seq.map (fun ch ->
            if (charMap[ch] >= 2)
            then s.Contains(sprintf "%c%c" ch ch)
            else false
        ) |> Seq.contains true
    if (isNice)
    then Nice
    else Naughty

let excludedStrings = [ "ab"; "cd"; "pq"; "xy" ]

// String checker #3 - check that no excluded strings occur
let excludedChecker: StringChecker = fun (s: string) (charMap: Map<char,int>) ->
    let isNaughty =
        excludedStrings |> List.map (fun excl -> s.Contains(excl)) |> List.contains true
    if (isNaughty)
    then Naughty
    else Nice

let naughtyOrNice: string -> StringCategorisation =
    createNaughtyOrNice ([vowelChecker; doubleChecker; excludedChecker])

In [13]:
let lines = File.ReadAllLines(stringInputFilePath)

let lineCount = lines |> Seq.length

let stringCategorisations = lines |> Seq.map naughtyOrNice

let isNice (sc: StringCategorisation) = sc = Nice

let niceCount = stringCategorisations |> Seq.filter (isNice) |> Seq.length

printfn "%d nice strings out of %d" niceCount lineCount

255 nice strings out of 1000


We can print the nice and naughty strings.

In [12]:
let printByCategory (category: StringCategorisation) (results: (StringCategorisation*string) seq) =
    printfn "%A" (
        Seq.zip stringCategorisations lines |> Seq.filter (fun pair ->
            let sc, _ = pair
            sc = category
        ) |> Seq.map (fun pair ->
            let _, line = pair
            line
        )
    )

In [14]:
let zippedResults = Seq.zip stringCategorisations lines

In [15]:
printByCategory Nice zippedResults

seq
  ["zwqadogmpjmmxijf"; "wrggegukhhatygfi"; "iuvrelxiapllaxbg";
   "feybgiimfthtplui"; ...]


In [16]:
printByCategory  Naughty zippedResults

seq
  ["sszojmmrrkwuftyv"; "isaljhemltsdzlum"; "fujcyucsrxgatisb";
   "qiqqlmcgnhzparyg"; ...]
